# <p style="text-align:center;"> **Entrega 4 - Ruta B**</p>
### <p style="text-align:center;"> *Sebastián y Vicente Ramírez*</p>
### <p style="text-align:center;"> **Apertura y limpieza de datasets**</p>
#### Librerías a usar

In [2]:
# !pip install prince # En caso de ser necesario
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.dummy import DummyRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
import itertools as it
import prince
import warnings

# Para una visualización más limpia (esta parte del código se corre con todo estructurado
# para evitar ignorar los warnings necesarios durante el proceso de desarrollo)
# warnings.filterwarnings("ignore")

# Configuración pandas
pd.set_option('display.max_columns', None)
pd.set_option("max_info_columns", 200)

#### Apertura de los dataset
> Incluye:
> > - grd_2023 (datos principales).
> > - CIE-10 (para diagnósticos).
> > - Hospitales (para decodificarlos).
> > - GRD (Para todos los diagnósticos).

In [3]:
# Lectura de archivos .csv y .xlsx
hospitales = pd.read_csv("./datasets/Hospitales.csv", header=None, sep="|")
cie = pd.read_excel("./datasets/CIE-10.xlsx")
grd = pd.read_parquet("./datasets/grd_2023.parquet")
grd_big = pd.read_parquet("./datasets/GRD.parquet")


display(grd.info())
display(grd_big.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039587 entries, 0 to 1039586
Data columns (total 16 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   COD_HOSPITAL         1039587 non-null  int64         
 1   CIP_ENCRIPTADO       1037952 non-null  float64       
 2   SEXO                 1039548 non-null  object        
 3   FECHA_NACIMIENTO     1039577 non-null  datetime64[ns]
 4   SERVICIO_SALUD       1038846 non-null  object        
 5   TIPO_INGRESO         1039532 non-null  object        
 6   FECHA_INGRESO        1039587 non-null  datetime64[ns]
 7   FECHAALTA            1039587 non-null  datetime64[ns]
 8   TIPOALTA             1039587 non-null  object        
 9   DIAGNOSTICO1         1039587 non-null  object        
 10  IR_29301_COD_GRD     1039557 non-null  float64       
 11  IR_29301_PESO        1039557 non-null  float64       
 12  IR_29301_SEVERIDAD   1039557 non-null  float64       
 1

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039587 entries, 0 to 1039586
Data columns (total 129 columns):
 #    Column                    Non-Null Count    Dtype         
---   ------                    --------------    -----         
 0    COD_HOSPITAL              1039587 non-null  int64         
 1    CIP_ENCRIPTADO            1039587 non-null  object        
 2    SEXO                      1039587 non-null  object        
 3    FECHA_NACIMIENTO          1039577 non-null  datetime64[ns]
 4    ETNIA                     1039587 non-null  object        
 5    PROVINCIA                 1039587 non-null  object        
 6    COMUNA                    1039587 non-null  object        
 7    NACIONALIDAD              1039587 non-null  object        
 8    PREVISION                 1039587 non-null  object        
 9    SERVICIO_SALUD            1039587 non-null  object        
 10   TIPO_PROCEDENCIA          1039587 non-null  object        
 11   TIPO_INGRESO              1039587 n

None

#### Transformaciones de columnas

In [4]:
# Para dejar como flotante CIP_ENCRIPTADO
def transformar(row):
    if row == "DESCONOCIDO":
        return np.nan
    return row

# Para decodificar los 10 diagnósticos
diagnostico = dict(zip(cie["Código"], cie["Descripción"]))

# Para decodificar hospitales
hospital = dict(zip(hospitales[0], hospitales[1]))

# Ajustar tipós de variable por columna para futuro merge
grd_big["CIP_ENCRIPTADO"] = grd_big["CIP_ENCRIPTADO"].apply(transformar)
grd_big["CIP_ENCRIPTADO"] = grd_big["CIP_ENCRIPTADO"].astype(float)
grd["FECHA_NACIMIENTO"] = grd["FECHA_NACIMIENTO"].astype('datetime64[ns]')
grd["FECHA_INGRESO"] = grd["FECHA_INGRESO"].astype('datetime64[ns]')
grd["FECHAALTA"] = grd["FECHAALTA"].astype('datetime64[ns]')

# Columans a eliminar por indice (Ver output anterior)
out = list(grd_big.columns[6:9]) + \
      list(grd_big.columns[10:11]) + \
      list(grd_big.columns[[4, 12, 13, 35]]) + \
      list(grd_big.columns[15:34]) + list(grd_big.columns[37:53]) + list(grd_big.columns[54:])

grd_big.drop(columns = out, axis = 1, inplace = True)

#### Merge de todos los datasets
> En el siguiente orden
> > 1. (grd  $\xleftrightarrow{merge}$  grd_main) $\rightarrow$ df | (Para tener todos los diagnósticos).
> > 2. df.map(diagnosticos) | para tener los diagnósticos en descripción. (hacerlo con las 10 columans)
> > 2. df.map(hospital) | Decodificar hospitales. Dataset completo.

In [5]:
df = grd.merge(grd_big, how = "left")

df['DESCRIPCION1'] = df["DIAGNOSTICO1"].map(diagnostico)
df.drop(columns = "DIAGNOSTICO1", axis = 1, inplace = True)

df["HOSPITAL"] = df["COD_HOSPITAL"].map(hospital)
df.drop(columns = "COD_HOSPITAL", axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041229 entries, 0 to 1041228
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   CIP_ENCRIPTADO       1039590 non-null  float64       
 1   SEXO                 1041190 non-null  object        
 2   FECHA_NACIMIENTO     1041219 non-null  datetime64[ns]
 3   SERVICIO_SALUD       1040488 non-null  object        
 4   TIPO_INGRESO         1041174 non-null  object        
 5   FECHA_INGRESO        1041229 non-null  datetime64[ns]
 6   FECHAALTA            1041229 non-null  datetime64[ns]
 7   TIPOALTA             1041229 non-null  object        
 8   IR_29301_COD_GRD     1041199 non-null  float64       
 9   IR_29301_PESO        1041199 non-null  float64       
 10  IR_29301_SEVERIDAD   1041199 non-null  float64       
 11  IR_29301_MORTALIDAD  1041199 non-null  float64       
 12  EDAD                 1041219 non-null  float64       
 1

#### Variable "REINGRESO"
> 1. Eliminar los CIP_ENCRIPTADO nulos
> > Si no se sabe que paciente entró, no es posible analizar de manera corercta tal variable. Además, son menos de 1500 valores nulos dentro de más de un millon de datos, por ende, no es una cantidad significativa.
> 2. Ordenar por fecha el dataset.
> > La función "duplicated" revisa el dataset por orden de registro, y el este no está ordenado por fecha de ingreso, por lo que aplicar directamente la función podría generar un reingreso antes de la primera atención.
> 3. Crear columna REINGRESO
> > Aplicando duplicated a la columna CIP_ENCRIPTADO, generando una columna de datos booleanos

In [6]:
# 1.
df.dropna(subset = ["CIP_ENCRIPTADO"], inplace = True)

# 2.
df.sort_values(by= "FECHA_INGRESO", ascending = True, inplace = True)

# 3.
df["REINGRESO"] = df["CIP_ENCRIPTADO"].duplicated()
df.reset_index(drop = True, inplace  = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039590 entries, 0 to 1039589
Data columns (total 18 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   CIP_ENCRIPTADO       1039590 non-null  float64       
 1   SEXO                 1039551 non-null  object        
 2   FECHA_NACIMIENTO     1039580 non-null  datetime64[ns]
 3   SERVICIO_SALUD       1038851 non-null  object        
 4   TIPO_INGRESO         1039535 non-null  object        
 5   FECHA_INGRESO        1039590 non-null  datetime64[ns]
 6   FECHAALTA            1039590 non-null  datetime64[ns]
 7   TIPOALTA             1039590 non-null  object        
 8   IR_29301_COD_GRD     1039560 non-null  float64       
 9   IR_29301_PESO        1039560 non-null  float64       
 10  IR_29301_SEVERIDAD   1039560 non-null  float64       
 11  IR_29301_MORTALIDAD  1039560 non-null  float64       
 12  EDAD                 1039580 non-null  float64       
 1

#### Tipo de ingreso
filtrado a urgencias

In [7]:
df = df[df["TIPO_INGRESO"] == "URGENCIA"]
df.reset_index(drop = True, inplace = True)

#### Filtrar por región metropolitana
> Para hacer un análisis más acotado. Incluir Provincias de:
> > - Santiago
> > - Talagante
> > - Cordillera
> > - Chacabuco
> > - Maipo
> > - Melipilla

In [8]:
provincias = ["SANTIAGO", "TALAGANTE", "CORDILLERA", "CHACABUCO", "MAIPO", "MELIPILLA"]
df = df[df["PROVINCIA"].isin(provincias)]
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174676 entries, 0 to 174675
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   CIP_ENCRIPTADO       174676 non-null  float64       
 1   SEXO                 174676 non-null  object        
 2   FECHA_NACIMIENTO     174672 non-null  datetime64[ns]
 3   SERVICIO_SALUD       174676 non-null  object        
 4   TIPO_INGRESO         174676 non-null  object        
 5   FECHA_INGRESO        174676 non-null  datetime64[ns]
 6   FECHAALTA            174676 non-null  datetime64[ns]
 7   TIPOALTA             174676 non-null  object        
 8   IR_29301_COD_GRD     174646 non-null  float64       
 9   IR_29301_PESO        174646 non-null  float64       
 10  IR_29301_SEVERIDAD   174646 non-null  float64       
 11  IR_29301_MORTALIDAD  174646 non-null  float64       
 12  EDAD                 174672 non-null  float64       
 13  DIAS_ESTANCIA 

#### Eliminación de las columans faltantes
>Tales como
> > - CIP_ENCRIPTADO
> > - FECHA_NACIMIENTO
> > - TIPO_INGRESO --> solo si filtramos por urgencia
> > - PROVINCIA
> > - FECHA_INGRESO
> > - FECHAALTA
> > - TIPOALTA
> > - IR_29301_COD_GRD
> > - IR_29301_PESO
> > - IR_29301_SEVERIDAD
> > - IR_29301_MORTALIDAD

In [9]:
out = ['CIP_ENCRIPTADO', 'FECHA_NACIMIENTO', 'TIPO_INGRESO', 'PROVINCIA', 'FECHA_INGRESO', 'FECHAALTA', 'TIPOALTA', 'IR_29301_COD_GRD', 'IR_29301_PESO', 'IR_29301_SEVERIDAD', 'IR_29301_MORTALIDAD']

df.drop(columns = out, axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174676 entries, 0 to 174675
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEXO            174676 non-null  object 
 1   SERVICIO_SALUD  174676 non-null  object 
 2   EDAD            174672 non-null  float64
 3   DIAS_ESTANCIA   174676 non-null  int64  
 4   DESCRIPCION1    174663 non-null  object 
 5   HOSPITAL        174676 non-null  object 
 6   REINGRESO       174676 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 8.2+ MB


#### Filtrar DIAS_ESTANCIA según recomendación
> Es decir, eliminar los outliers de más de 15 días de estancia (y los pacientes sin estancia)

In [10]:
df = df[(df["DIAS_ESTANCIA"] > 0) & (df["DIAS_ESTANCIA"] <= 15)]
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138667 entries, 0 to 138666
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEXO            138667 non-null  object 
 1   SERVICIO_SALUD  138667 non-null  object 
 2   EDAD            138665 non-null  float64
 3   DIAS_ESTANCIA   138667 non-null  int64  
 4   DESCRIPCION1    138654 non-null  object 
 5   HOSPITAL        138667 non-null  object 
 6   REINGRESO       138667 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 6.5+ MB


#### Manejo de nulos.
> Se revisarán los nulos de todas las variables:
> > - SEXO
> > - SERVICIO_SALUD
> > - EDAD
> > - DIAS_ESTANCIA
> > - DESCRIPCION1
> > - HOSPITAL
> > - REINGRESO

In [11]:
var = ["SEXO", "SERVICIO_SALUD", "EDAD", "DIAS_ESTANCIA", "DESCRIPCION1", "HOSPITAL", "REINGRESO"]
nulos = 0
for i in var:
    nul = round((df[i].isnull().sum()/len(df))*100, 3)
    nulos += nul
    print(f'Variable: {i}')
    print(f'Número de nulos: {df[i].isnull().sum()}')
    print(f'Porcentaje de nulos: {nul}%')
    print('\n---------------------------------------------------------------------------\n')
print(f"Porcentaje acumulado de nulos: {nulos}%")


Variable: SEXO
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: SERVICIO_SALUD
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: EDAD
Número de nulos: 2
Porcentaje de nulos: 0.001%

---------------------------------------------------------------------------

Variable: DIAS_ESTANCIA
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: DESCRIPCION1
Número de nulos: 13
Porcentaje de nulos: 0.009%

---------------------------------------------------------------------------

Variable: HOSPITAL
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: REINGRESO
Número de nulos: 0
Porcentaje de nulos: 0.0%

----------------------------------------------------------------------

Como el porcentaje de nulos es de un 0.01% de los datos, se pueden eliminar sin problema.

In [12]:
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138652 entries, 0 to 138651
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEXO            138652 non-null  object 
 1   SERVICIO_SALUD  138652 non-null  object 
 2   EDAD            138652 non-null  float64
 3   DIAS_ESTANCIA   138652 non-null  int64  
 4   DESCRIPCION1    138652 non-null  object 
 5   HOSPITAL        138652 non-null  object 
 6   REINGRESO       138652 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 6.5+ MB


### <p style="text-align:center;"> **Análisis de variables**</p>:

Como todas las variables son categóricas, las métricas principales son:
> - Frecuencia Absoluta
> - Frecuencia Relativa
> - Moda
> - Cantidad de categorías
> - Pie Chart:
> > Este útlimo solo si hay <5 categorías. En caso contrario, es ilegible.

#### DESCRIPCION1

In [18]:
medidas = df.groupby("DESCRIPCION1")["DESCRIPCION1"].count().to_frame(name = "Frecuencia Absoluta").sort_values(by = "Frecuencia Absoluta", ascending = False)
medidas["Frecuencia Relativa"] = medidas["Frecuencia Absoluta"]*100 / sum(medidas["Frecuencia Absoluta"])
display(medidas.head(10))
print(f'n°Categorías: {len(medidas)}')
print(f'Moda:         {medidas.index[0]}')

,Frecuencia Absoluta,Frecuencia Relativa
DESCRIPCION1,,
"Apendicitis aguda, otra y no especificada",4380,3.158988
Cálculo de la vesícula biliar con colecistitis aguda,2705,1.950928
Nefritis tubulointersticial aguda,2451,1.767735
Neumonía debida a virus sincitial respiratorio,2165,1.561463
Estado asmático,2134,1.539105
"Neumonía, no especificada",1908,1.376107
"Infección de vías urinarias, sitio no especificado",1882,1.357355
"Neumonía viral, no especificada",1661,1.197963
Diabetes mellitus tipo 2 con complicaciones circulatorias periféricas,1640,1.182817


n°Categorías: 4115
Moda:         Apendicitis aguda, otra y no especificada


#### Hospitales

In [19]:
medidas = df.groupby("HOSPITAL")["HOSPITAL"].count().to_frame(name = "Frecuencia Absoluta").sort_values(by = "Frecuencia Absoluta", ascending = False)
medidas["Frecuencia Relativa"] = medidas["Frecuencia Absoluta"]*100 / sum(medidas["Frecuencia Absoluta"])
display(medidas.head(10))
print(f'n°Categorías: {len(medidas)}')
print(f'Moda:         {medidas.index[0]}')

,Frecuencia Absoluta,Frecuencia Relativa
HOSPITAL,,
"Complejo Hospitalario Dr. Sótero del Río (Santiago, Puente Alto)",17976,12.964833
"Hospital Dr. Félix Bulnes Cerda (Santiago, Quinta Normal)",11181,8.064074
"Hospital Barros Luco Trudeau (Santiago, San Miguel)",10275,7.410640
Hospital Clínico Metropolitano El Carmen Doctor Luis Valentín Ferrada,9877,7.123590
"Complejo Hospitalario San José (Santiago, Independencia)",8270,5.964573
Hospital de Urgencia Asistencia Pública Dr. Alejandro del Río,7842,5.655887
Hospital Clínico Metropolitano La Florida Dra. Eloisa Díaz Inzunza,7804,5.628480
"Hospital San Juan de Dios (Santiago, Santiago)",7564,5.455385
Hospital Padre Alberto Hurtado (San Ramón),7202,5.194299


n°Categorías: 67
Moda:          Complejo Hospitalario Dr. Sótero del Río (Santiago, Puente Alto)


####